In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id:  16754


In [12]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [15]:
#get only the shotsData
strings = scripts[1].string

In [16]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22477832\\x22,\\x22minute\\x22\\x3A\\x2218\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.904000015258789\\x22,\\x22Y\\x22\\x3A\\x220.5879999923706055\\x22,\\x22xG\\x22\\x3A\\x220.019936958327889442\\x22,\\x22player\\x22\\x3A\\x22Joel\\x20Ward\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x22510\\x22,\\x22situation\\x22\\x3A\\x22SetPiece\\x22,\\x22season\\x22\\x3A\\x222021\\x22,\\x22shotType\\x22\\x3A\\x22Head\\x22,\\x22match_id\\x22\\x3A\\x2216751\\x22,\\x22h_team\\x22\\x3A\\x22Crystal\\x20Palace\\x22,\\x22a_team\\x22\\x3A\\x22Manchester\\x20United\\x22,\\x22h_goals\\x22\\x3A\\x221\\x22,\\x22a_goals\\x22\\x3A\\x220\\x22,\\x22date\\x22\\x3A\\x222022\\x2D05\\x2D22\\x2015\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Conor\\x20Gallagher\\x22,\\x22lastAction\\x22\\x3A\\x22Aerial\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22477835\\x22,\\x22minute\\x22\\x3A\\x2225\\x

In [6]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [7]:
data

{'h': [{'id': '477832',
   'minute': '18',
   'result': 'MissedShots',
   'X': '0.904000015258789',
   'Y': '0.5879999923706055',
   'xG': '0.019936958327889442',
   'player': 'Joel Ward',
   'h_a': 'h',
   'player_id': '510',
   'situation': 'SetPiece',
   'season': '2021',
   'shotType': 'Head',
   'match_id': '16751',
   'h_team': 'Crystal Palace',
   'a_team': 'Manchester United',
   'h_goals': '1',
   'a_goals': '0',
   'date': '2022-05-22 15:00:00',
   'player_assisted': 'Conor Gallagher',
   'lastAction': 'Aerial'},
  {'id': '477835',
   'minute': '25',
   'result': 'SavedShot',
   'X': '0.9030000305175782',
   'Y': '0.6619999694824219',
   'xG': '0.07834828644990921',
   'player': 'Wilfried Zaha',
   'h_a': 'h',
   'player_id': '522',
   'situation': 'OpenPlay',
   'season': '2021',
   'shotType': 'LeftFoot',
   'match_id': '16751',
   'h_team': 'Crystal Palace',
   'a_team': 'Manchester United',
   'h_goals': '1',
   'a_goals': '0',
   'date': '2022-05-22 15:00:00',
   'player

In [8]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [9]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [10]:
df

,x,y,xG,result,team
0,0.904000015258789,0.5879999923706055,0.019936958327889442,MissedShots,Crystal Palace
1,0.9030000305175782,0.6619999694824219,0.07834828644990921,SavedShot,Crystal Palace
2,0.9030000305175782,0.6890000152587891,0.05674053356051445,SavedShot,Crystal Palace
3,0.850999984741211,0.4759999847412109,0.07944738864898682,Goal,Crystal Palace
4,0.7909999847412109,0.5479999923706055,0.05098433420062065,MissedShots,Crystal Palace
5,0.885,0.56,0.06916061788797379,MissedShots,Crystal Palace
6,0.8719999694824219,0.40099998474121096,0.07951131463050842,BlockedShot,Manchester United
7,0.7340000152587891,0.45099998474121095,0.01853969879448414,BlockedShot,Manchester United
8,0.865,0.555,0.07512732595205307,SavedShot,Manchester United
9,0.7959999847412109,0.75,0.0326865129172802,BlockedShot,Manchester United
